In [145]:
import pandas as pd
import pickle
from pathlib import Path
import plotly.express as px
px.set_mapbox_access_token(open("mapbox_token.txt").read())

In [146]:
Path(Path.cwd(), "raw_data", "wind_power", "operational_list.pkl")

PosixPath('/Users/jamesmoro/Documents/Python/UK_Net_Zero/raw_data/wind_power/operational_list.pkl')

In [147]:
def return_substring(raw_string:str, substring:str) -> str:
    start_idx = raw_string.find(substring) + len(substring)
    return raw_string[start_idx:].strip()

def generate_project_info(raw_list:list) -> dict:
    project_info_dict = {}
    split_list = raw_list.split("\n")
    project_info_dict['Project Name'] = return_substring(split_list[0], "Name")
    project_info_dict['Location'] = return_substring(split_list[1], "Location")
    project_info_dict['County'] = return_substring(split_list[2] + " " + split_list[3], "Co.")
    project_info_dict['Turbine Capacity'] = float(return_substring(split_list[4], "Capacity"))
    project_info_dict['Number of Turbines'] = return_substring(split_list[5], "Turbines")
    project_info_dict['Project Capacity (MW)'] = float(return_substring(split_list[6], "(MW)"))
    project_info_dict['Developer'] = return_substring(split_list[7], "Developer")
    project_info_dict['Owner'] = return_substring(split_list[8], "Owner")
    project_info_dict['Local Planning Authority'] = return_substring(split_list[9], "Authority")
    
    end_idx = split_list[11].find("Longitude (decimal)")
    project_info_dict['Latitude'] = float(split_list[11][:end_idx].strip())
    
    project_info_dict['Longitude'] = float(return_substring(split_list[11], "decimal)"))
    
    temp_string = split_list[12] + " " + split_list[13]
    end_idx = temp_string.find("Project")
    project_info_dict['Type'] = temp_string[:end_idx].replace("Type", "").strip()
    
    project_info_dict['Project Status'] = return_substring(split_list[13], "Status")
    
    return project_info_dict

# Operational

In [148]:
operational_pickle = Path(Path.cwd(), "raw_data", "wind_power", "operational_list.pkl")
with open(operational_pickle, 'rb') as operational:
    operational_list = pickle.load(operational)

In [149]:
info_dict_list = []
errors = []
for i, _ in enumerate(operational_list):
    try:
        info_dict_list.append(generate_project_info(operational_list[i]))
    except (IndexError, ValueError) as e:
        errors.append(i)
        continue

In [150]:
df_operational = pd.DataFrame(info_dict_list)

In [151]:
fig = px.scatter_mapbox(df_operational, 
                        lat="Latitude", 
                        lon="Longitude",     
                        color="Project Capacity (MW)", 
                        hover_data = ["Project Name", "Number of Turbines", "Project Capacity (MW)"],
                  color_continuous_scale=px.colors.cyclical.IceFire, 
                  size_max=15, 
                  zoom=3)
fig.show()

# Consented

In [152]:
consented_pickle = Path(Path.cwd(), "raw_data", "wind_power", "consented_list.pkl")
with open(consented_pickle, 'rb') as consented:
    consented_list = pickle.load(consented)

In [153]:
info_dict_list = []
errors = []
for i, _ in enumerate(consented_list):
    try:
        info_dict_list.append(generate_project_info(consented_list[i]))
    except (IndexError, ValueError) as e:
        errors.append(i)
        continue

In [154]:
df_consented = pd.DataFrame(info_dict_list)

In [155]:
fig = px.scatter_mapbox(df_consented, 
                        lat="Latitude", 
                        lon="Longitude",     
                        color="Project Capacity (MW)", 
                        hover_data = ["Project Name", "Number of Turbines", "Project Capacity (MW)"],
                  color_continuous_scale=px.colors.cyclical.IceFire, 
                  size_max=15, 
                  zoom=3)
fig.show()

# Under construction

In [156]:
under_construction_pickle = Path(Path.cwd(), "raw_data", "wind_power", "under_construction_list.pkl")
with open(under_construction_pickle, 'rb') as under_construction:
    under_construction_list = pickle.load(under_construction)

In [157]:
info_dict_list = []
errors = []
for i, _ in enumerate(under_construction_list):
    try:
        info_dict_list.append(generate_project_info(under_construction_list[i]))
    except (IndexError, ValueError) as e:
        errors.append(i)
        continue

In [158]:
df_under_construction = pd.DataFrame(info_dict_list)

In [159]:
fig = px.scatter_mapbox(df_under_construction, 
                        lat="Latitude", 
                        lon="Longitude",     
                        color="Project Capacity (MW)", 
                        hover_data = ["Project Name", "Number of Turbines", "Project Capacity (MW)"],
                  color_continuous_scale=px.colors.cyclical.IceFire, 
                  size_max=15, 
                  zoom=3)
fig.show()